In [1]:
import numpy as np
import sympy as sm
#measurements: http://hyperphysics.phy-astr.gsu.edu/hbase/vision/eyescal.html

In [2]:
#focal length
def f(n1,n2,r):
    return r/(n2-n1)

#translation
def T(d):
    return sm.Matrix([[1,d],[0,1]])

#refraction
def R(n1,n2,f):
    return sm.Matrix([[1,0],[-(1/n2*f), n1/n2]])

#thin lens
def L_thin(f1,f2):
    return sm.Matrix([[1,0],[-(1/f1 + 1/f2), 1]])

def L_thick(R1,T,R2):
    mult = R2*T
    return R1*mult

In [7]:
#section 1: air
d0 = sm.symbols('d0')

#section 2: cornea (thick lens, constant)
dc = .449e-3
Rc1 = 7.259e-3
Rc2 = 5.585e-3
nair = 1
nc = 1.376
nah = 1.336
fc1 = f(nair,nc,Rc1)
fc2 = f(nc,nah,Rc2)

#section 3 (translation through AH, dah constant)
dah = 2.794e-3

#4: lens (assume nl constant (it varies w/ depth into lens); dl, Rl1, Rl2 are what change here)
# really it's a series of thin lenses but that doesn't sound fun, treat as thick lens
dl, Rl1, Rl2 = sm.symbols('dl, Rl1, Rl2')
dl = 4e-3 #assume constant thickness to simplify?
nl = 1.396 #nl ranges from 1.406 - 1.386
nvh = 1.337 #vitreous humor
fl1 = f(nah, nl, Rl1)
fl2 = f(nl,nvh, Rl2)
#considering how these focal lengths change? 1/f=1/d0+1/di
#fl1,fl2 = sm.symbols('fl1, fl2')

#5: vitreous humor (translation)
dvh = 24e-3 - dc - dah - dl #maybe assume constant if it's too complicated  - yes

In [67]:
#matrices 
T1 = T(d0)
L2_1 = R(nair,nc,fc1)
L2_2 = T(dc)
L2_3 = R(nc,nah,fc2)
T3 = T(dah)
L4_1 = R(nah, nl, fl1)
L4_2 = T(dl)
L4_3 = R(nl, nvh, fl2)
T5 = T(dvh)
#system matrix for light entering the eye:
# don't want T5(end of lens to retina) bc we're trying to find that focal length (to test)
#bc these are the focal lengths we're trying to find
M = L4_3*L4_2*L4_1*T3*L2_3*L2_2*L2_1*T1
M

Matrix([
[                                                                     1.0005900755178 - 0.0477672074512753*Rl1,                                                                                                                                       -0.000115459445407935*Rl1 + d0*(1.0005900755178 - 0.0477672074512753*Rl1) + 0.00528318084636646],
[-11.9418018628188*Rl1*(0.0507080105979743*Rl2 + 1.04412864622289) + 12.6844830383961*Rl2 + 0.0899912261381107, -0.0288648613519837*Rl1*(0.0507080105979743*Rl2 + 1.04412864622289) + 0.0669748975871412*Rl2 + d0*(-11.9418018628188*Rl1*(0.0507080105979743*Rl2 + 1.04412864622289) + 12.6844830383961*Rl2 + 0.0899912261381107) + 0.747977233189216]])

In [69]:
#find focal length: send collumated light in at alpha=0
#this should always be a couple mm less than 24mm
h1, a1 = sm.symbols('h1, a1')
r_in = sm.Matrix([h1,0])
r_out = M*r_in
h2, a2 = r_out
#a2 = h2/f
f_in = h2/a2 #focal length for light entering eye (=di)
print('focal length of eye for light going in as a function of Rl1,Rl2: f_in = ',f_in)
#f_in is a list, just paste it here for simplicity
f_in_eq = lambda Rl1, Rl2: (1.0005900755178 - 0.0477672074512753*Rl1)/(-11.9418018628188*Rl1*(0.0507080105979743*Rl2 + 1.04412864622289) + 12.6844830383961*Rl2 + 0.0899912261381107)

f_in_eq(8.672e-3,6.328e-3), dvh #not bad, except for the order of magnitude

focal length of eye for light going in as a function of Rl1,Rl2: f_in =  (1.0005900755178 - 0.0477672074512753*Rl1)/(-11.9418018628188*Rl1*(0.0507080105979743*Rl2 + 1.04412864622289) + 12.6844830383961*Rl2 + 0.0899912261381107)


(16.10688529615115, 0.016756999999999998)

In [63]:
#want to find relationship between lens radii and object disance, so want focal length of light leaving eye
#again, no T1
M_out = L2_1*L2_2*L2_3*T3*L4_1*L4_2*L4_3*T5
#system matrix for light leaving the eye:
M_out

Matrix([
[-0.0388798259533964*Rl1 + 12.6770026494936*Rl2*(0.0071167945479003 - 0.000155519303813586*Rl1) + 1.00004692486901, -0.000813891403653469*Rl1 + 0.212428533397564*Rl2*(0.0071167945479003 - 0.000155519303813586*Rl1) + 0.0241886353767756],
[   -8.93827020490006*Rl1 + 12.6770026494936*Rl2*(0.716739422687434 - 0.0357530808196002*Rl1) + 0.0619207656617446,        -0.187109409697977*Rl1 + 0.212428533397564*Rl2*(0.716739422687434 - 0.0357530808196002*Rl1) + 0.749405769375398]])

In [72]:
#now find d0 = consider focal length when sending light out
#except now do we need to use a different r_in? since we're trying to take the opposite path of incoming light
r_in = sm.Matrix([h1,0])
r_out = M_out*r_in
h2,a2 = r_out
f_out = h2/a2 #=d0

print('d0 as a function of Rl1, Rl2 = ',f_out)
#f_out is a list, just paste it here for simplicity
f_out_eq = lambda Rl1, Rl2: (-0.0388798259533964*Rl1 + 12.6770026494936*Rl2*(0.0071167945479003 - 0.000155519303813586*Rl1) + 1.00004692486901)/(-8.93827020490006*Rl1 + 12.6770026494936*Rl2*(0.716739422687434 - 0.0357530808196002*Rl1) + 0.0619207656617446)

Rl1=8.672e-3; Rl2=6.328e-3
d0_sol_2(Rl1,Rl2, 20e-3),d0_sol_1(Rl1,Rl2, di_sol_1)
print('\n testing focal length (ie. distance to object from eye) for different lens radii:')
print('focal length for light leaving eye with lens at radii %.6f, %.6f: f_out = %.8f' %(Rl1,Rl2,f_out_eq(8.672e-3,6.328e-3)))
Rl1=8.57e-3; Rl2=6.17e-3
print('focal length for light leaving eye with lens at radii %.6f, %.6f: f_out = %.8f' %(Rl1,Rl2,f_out_eq(Rl1,Rl2) ))
Rl1=7e-3; Rl2=6e-3
print('focal length for light leaving eye with lens at radii %.6f, %.6f: f_out = %.8f' %(Rl1,Rl2,f_out_eq(Rl1,Rl2) ))
Rl1=4e-3; Rl2=2e-3
print('focal length for light leaving eye with lens at radii %.6f, %.6f: f_out = %.8f' %(Rl1,Rl2,f_out_eq(Rl1,Rl2) ))


d0 as a function of Rl1, Rl2 =  (-0.0388798259533964*Rl1 + 12.6770026494936*Rl2*(0.0071167945479003 - 0.000155519303813586*Rl1) + 1.00004692486901)/(-8.93827020490006*Rl1 + 12.6770026494936*Rl2*(0.716739422687434 - 0.0357530808196002*Rl1) + 0.0619207656617446)

 testing focal length (ie. distance to object from eye) for different lens radii:
focal length for light leaving eye with lens at radii 0.008672, 0.006328: f_out = 23.88438647
focal length for light leaving eye with lens at radii 0.008570, 0.006170: f_out = 24.18617580
focal length for light leaving eye with lens at radii 0.007000, 0.006000: f_out = 18.57580355
focal length for light leaving eye with lens at radii 0.004000, 0.002000: f_out = 22.55651497


assumptions/approximations:
- only considering one eye
- small angle approx in focal length calculation
- assuming constant nl
- assuming constant lens thickness (dl), and thus constant vh thickness (dvh)
- only considering h and alpha, (2 dimensions)
- not considering relationship between dl and lens radii (i think this is a big one)